In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib as ta
import os
import mplfinance as mpf
import numpy as np
from dotenv import load_dotenv

In [ ]:
def get_time_stamp():
    return datetime.now().strftime('%Y%m%d%H%M%S')

In [ ]:
load_dotenv(override=True)
output_dir = os.getenv("OUTPUT_DIRECTORY")
directory = 'bitget-' + get_time_stamp()
if output_dir is not None :
    directory = os.path.join(output_dir, directory)
os.makedirs(directory, exist_ok=True)

In [ ]:
def get_pairs() :
    return requests.get('https://api.bitget.com/api/v2/spot/public/symbols').json()

In [ ]:
def get_ticker(symbol, granularity, time_from, time_to) :
    request_str = 'https://api.bitget.com/api/v2/spot/market/candles' + '?symbol=' + symbol + '&granularity=' + granularity + '&startTime=' + str(time_from) + '&endTime=' + str(time_to) + '&limit=1000'
    return requests.get(request_str).json()

In [ ]:
width_ratio = 0.2

now = datetime.now()
past = now - timedelta(days=5*365)
time_from_ms = int(past.timestamp() * 1000)
time_to_ms = int(now.timestamp() * 1000)
time_frame = '1Dutc'

In [ ]:
pairs = get_pairs()
symbols = [item['symbol'] for item in pairs['data'] if 'symbol' in item and item['symbol'].endswith('USDT')]
remove_list = ['BTCUSDT', 'USDCUSDT', 'USDPUSDT', 'USTCUSDT', 'FDUSDUSDT', 'TUSDUSDT', 'AEURUSDT', 'DAIUSDT', 'USDEUSDT', 'DEUSDUSDT', 'PYUSDUSDT', 'WUSDUSDT', 'WBTCUSDT', 'WETHUSDT', 'WEETHUSDT', 'STETHUSDT', 'USDSUSDT', 'TUSDTUSDT', 'SPELLUSDT', 'SSVUSDT']
symbols = [item for item in symbols if item not in remove_list]
symbols.sort()
print(symbols)

In [ ]:
def get_df_from_ticker(ticker_name, granularity, time_from, time_to) :
    ticker = get_ticker(ticker_name + 'USDT', granularity, time_from, time_to)
    if len(ticker['data']) == 0 :
        return None
    df = pd.DataFrame(ticker['data'], columns=[
    'Timestamp',
    'Open',
    'High',
    'Low',
    'Close',
    'Volume Base',
    'Volume USDT',
    'Volume Quote'])
    df['Date'] = pd.to_datetime(pd.to_numeric(df['Timestamp']), unit='ms')
    df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df['High'] = pd.to_numeric(df['High'], errors='coerce')
    df['Low'] = pd.to_numeric(df['Low'], errors='coerce')
    df['Volume'] = pd.to_numeric(df['Volume USDT'], errors='coerce')
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    return df

In [ ]:
df_btc_usdt = get_df_from_ticker('BTC', time_frame, time_from_ms, time_to_ms)

In [ ]:
def check_first_candle(df) :
    first_high = df['High'].iloc[0]
    first_low = df['Low'].iloc[0]
    if first_high/first_low > 2 :
        return True
    else :
        return False

In [ ]:
def get_indicator(df_final) :
    df_final['RSI_14'] = ta.RSI(df_final['Close'], timeperiod=14)

    df_final['OBV'] = ta.OBV(df_final['Close'], df_final['Volume'])

    slowk, slowd = ta.STOCH(df_final['High'], df_final['Low'], df_final['Close'],
                           fastk_period=5,
                           slowk_period=3,
                           slowd_period=3)
    df_final['Stoch_%K'] = slowk
    df_final['Stoch_%D'] = slowd
    return df_final

In [ ]:
def get_history_btc_pair(ticker_name, df_coin_usdt, df_btc_usdt_local) :
    ticker_name_low = ticker_name.lower()
    df_merged = pd.merge(df_coin_usdt, df_btc_usdt_local, left_index=True, right_index=True, suffixes=('_' + ticker_name_low, '_btc'))
    df_merged['Open_' + ticker_name_low +'_btc'] = df_merged['Open_' + ticker_name_low] / df_merged['Open_btc']
    df_merged['High_' + ticker_name_low +'_btc'] = df_merged['High_' + ticker_name_low] / df_merged['High_btc']
    df_merged['Low_' + ticker_name_low + '_btc'] = df_merged['Low_' + ticker_name_low] / df_merged['Low_btc']
    df_merged['Close_'+ ticker_name_low +'_btc'] = df_merged['Close_' + ticker_name_low] / df_merged['Close_btc']
    df_merged['Volume_'+ ticker_name_low +'_btc'] = df_merged['Volume_' + ticker_name_low]

    df_final = df_merged[['Open_' + ticker_name_low +'_btc', 'High_' + ticker_name_low +'_btc', 'Low_' + ticker_name_low + '_btc', 'Close_' + ticker_name_low +'_btc', 'Volume_' + ticker_name_low +'_btc']].copy()
    df_final.rename(columns={
        'Open_' + ticker_name_low +'_btc': 'Open',
        'High_' + ticker_name_low +'_btc': 'High',
        'Low_' + ticker_name_low + '_btc': 'Low',
        'Close_' + ticker_name_low +'_btc': 'Close',
        'Volume_' + ticker_name_low +'_btc': 'Volume'
    }, inplace=True)

    mask = df_final['High'] < df_final['Low']
    df_final.loc[mask, ['High', 'Low']] = df_final.loc[mask, ['Low', 'High']].values

    if check_first_candle(df_final) :
        df_final.drop(index=df_final.index[0], inplace=True)

    df_final['Return'] = df_final['Close'].pct_change()
    df_final['Cum Return'] = (1 + df_final['Return']).cumprod() - 1

    df_final['RSI_14'] = ta.RSI(df_final['Close'], timeperiod=14)

    df_final['OBV'] = ta.OBV(df_final['Close'], df_final['Volume'])

    slowk, slowd = ta.STOCH(df_final['High'], df_final['Low'], df_final['Close'],
                           fastk_period=5,
                           slowk_period=3,
                           slowd_period=3)
    df_final['Stoch_%K'] = slowk
    df_final['Stoch_%D'] = slowd

    len_data = len(df_merged['Close_' + ticker_name_low])
    if len_data > 2 :
        min_time_period = min(len_data-1, 90)
        df_final['Beta'] = ta.BETA(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)
        df_final['Correl'] = ta.CORREL(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)

    df_final['Change1D'] = df_final['Close'].pct_change() * 100
    df_final['Change7D'] = df_final['Close'].pct_change(periods=7) * 100
    df_final['Change30D'] = df_final['Close'].pct_change(periods=30) * 100

    # df_final['EMA_13'] = df_final['Close'].ewm(span=13, adjust=False).mean()
    # df_final['EMA_21'] = df_final['Close'].ewm(span=21, adjust=False).mean()

    return df_final

In [ ]:
def resampling_weekly(df_daily) :
    return df_daily.resample('W').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

In [ ]:
def show_candlestick(tickername, tickerbase, df_input, timeframe_str) :
    df = df_input.tail(100) if len(df_input) >= 100 else df_input
    if timeframe_str == 'Weekly' :
        timeframe_id_str = 'Mingguan'
    else :
        timeframe_id_str = 'Harian'
    mpf.plot(df, type='candle', style='binance', volume=True, title=f'Pergerakan Harga {timeframe_id_str} {tickername} terhadap {tickerbase}', savefig=dict(fname=os.path.join(directory, f'Candlestick-{timeframe_str}-{tickername}-{tickerbase}-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight'))

In [ ]:
def show_chart(df_input, tickername, tickerbase, timeframe_str) :
    df = df_input.tail(100) if len(df_input) >= 100 else df_input
    time_now = datetime.now().strftime('%d-%m-%Y %H:%M:%S')
    if timeframe_str == 'Weekly' :
        timeframe_id_str = 'Mingguan'
    else :
        timeframe_id_str = 'Harian'

    # Membuat figure dan dua subplot yang berbagi sumbu x
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(14, 10))

    # Pergerakan Harga
    ax1.plot(df.index, df['Close'], label=f'{tickername}/{tickerbase}', color='blue')
    # ax1.plot(df.index, df['EMA_13'], linewidth=0.5, label='EMA 13', color='yellow')
    # ax1.plot(df.index, df['EMA_21'], linewidth=0.5, label='EMA 21', color='orange')
    ax1.set_title(f'Pergerakan Harga {timeframe_id_str} {tickername} terhadap {tickerbase}')
    ax1.set_ylabel(f'Harga ({tickerbase})')
    ax1.legend()
    ax1.grid(True)
    ax1.text(0.99, 1.05, time_now,
         transform=ax1.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax1.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax1.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # RSI
    ax2.plot(df.index, df['RSI_14'], label=f'RSI {tickername}/{tickerbase}', color='orange')
    ax2.axhline(70, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax2.axhline(30, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax2.set_title(f'Grafik RSI {timeframe_id_str} {tickername} terhadap {tickerbase}')
    # ax2.set_xlabel('Tanggal')
    ax2.set_ylabel('RSI')
    ax2.legend()
    ax2.grid(True)
    ax2.text(0.99, 1.05, time_now,
         transform=ax2.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax2.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax2.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Stochastic
    ax3.plot(df.index, df['Stoch_%K'], label='%K', color='blue')
    ax3.plot(df.index, df['Stoch_%D'], label='%D', color='orange')
    ax3.axhline(80, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax3.axhline(20, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax3.set_title(f'Grafik Stochastic {timeframe_id_str} {tickername} terhadap {tickerbase}')
    # ax3.set_xlabel('Tanggal')
    ax3.set_ylabel('Nilai')
    ax3.legend()
    ax3.grid(True)
    ax3.text(0.99, 1.05, time_now,
         transform=ax3.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax3.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax3.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # OBV
    ax4.plot(df.index, df['OBV'], label='OBV', color='blue')
    ax4.set_title(f'Grafik OBV {timeframe_id_str} {tickername} terhadap {tickerbase}')
    ax4.set_xlabel('Tanggal')
    ax4.set_ylabel('Nilai')
    ax4.legend()
    ax4.grid(True)
    ax4.text(0.99, 1.05, time_now,
         transform=ax4.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax4.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax4.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Menyesuaikan tata letak agar tidak saling tumpang tindih
    plt.tight_layout()
    # plt.show()
    plt.savefig(os.path.join(directory, 'Chart-' + timeframe_str + '-' + tickername + '-' + tickerbase + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_rsi(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['RSI_14'], color='blue')
    plt.axhline(70, linestyle='--', alpha=0.5, color='red', label='Overbought (70)')
    plt.axhline(30, linestyle='--', alpha=0.5, color='green', label='Oversold (30)')
    plt.title('Nilai RSI Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('RSI')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'RSI-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_beta(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Beta'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Beta = 0')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Beta = 1')
    plt.title('Nilai Beta (Volatilitas Harga Token Terhadap BTC)')
    plt.xlabel('Ticker')
    plt.ylabel('Beta')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Beta-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_correl(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Correl'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Correlation = 0')
    plt.axhline(-1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = -1')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = 1')
    plt.title('Nilai Pearson Correlation Coefficient Terhadap BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Correlation')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Correlation-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_change(df, chart_title) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='Change = 0%')
    plt.title(chart_title)
    plt.xlabel('Ticker')
    plt.ylabel('Perubahan (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, chart_title + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_momentum(df, chart_title) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.title(chart_title)
    plt.xlabel('Ticker')
    plt.ylabel('Momentum')
    plt.xticks(rotation=90)
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, chart_title + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_reversal(df, chart_title) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.title(chart_title)
    plt.xlabel('Ticker')
    plt.ylabel('Reversal')
    plt.xticks(rotation=90)
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, chart_title + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_return_mean(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Rata-Rata Return Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Return Mean (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Rata-Rata Return-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_return_std(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Volatilitas Return Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Tingkat Volatilitas')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Volatilitas-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_sharpe_ratio(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Sharpe Ratio Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Sharpe Ratio')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Sharpe Ratio-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_adj_return_per_volatility(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Adjusted Return Per Unit Volatility Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Adjusted Return Per Unit Volatility')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Adjusted Return Per Unit Volatility-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chart_sortino_ratio(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Sortino Ratio Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Sortino Ratio')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Sortino Ratio-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def calculate_downside_std(returns) :
    downside_returns = returns[returns < 0]
    if len(downside_returns) < 2 :
        return None
    return np.sqrt((downside_returns ** 2).sum()/(len(downside_returns) - 1))

In [ ]:
def show_multiple_chart(tickers, df_btc_usdt_local, is_show_chart=False) :
    rsi_list = []
    beta_list = []
    correl_list = []
    latest_change_1d_list = []
    latest_change_7d_list = []
    latest_change_30d_list = []
    # return_mean_list = []
    # return_std_list = []
    sharpe_ratio_list = []
    # adj_return_per_volatility_list = []
    sortino_ratio_list = []
    momentum_list = []
    reversal_list = []

    tickers.sort()

    for i, ticker in enumerate(tickers) :
        print('Processing ' + ticker + '...')
        df_coin_usdt = get_df_from_ticker(ticker, time_frame, time_from_ms, time_to_ms)
        if df_coin_usdt is None or df_coin_usdt.empty :
            print('Skipped ' + ticker + '...')
            continue
        df_btc_pair = get_history_btc_pair(ticker, df_coin_usdt, df_btc_usdt_local)

        if len(df_btc_pair['Close']) < 14 :
            print('Skipped ' + ticker + '...')
            continue

        if is_show_chart :
            show_chart(df_btc_pair, ticker, 'BTC', 'Daily')
            show_candlestick(ticker, 'BTC', df_btc_pair, 'Daily')

        if 'RSI_14' in df_btc_pair.columns :
            latest_rsi = df_btc_pair['RSI_14'].iloc[-1]
            if not pd.isna(latest_rsi) :
                rsi_list.append({'Label': ticker, 'RSI_14': latest_rsi})

        if 'Beta' in df_btc_pair.columns :
            latest_beta = df_btc_pair['Beta'].iloc[-1]
            if not pd.isna(latest_beta) :
                beta_list.append({'Label': ticker, 'Beta': latest_beta})

        if 'Correl' in df_btc_pair.columns :
            latest_correl = df_btc_pair['Correl'].iloc[-1]
            if not pd.isna(latest_correl) :
                correl_list.append({'Label': ticker, 'Correl': latest_correl})

        latest_change_1d = df_btc_pair['Change1D'].iloc[-1]
        latest_change_7d = df_btc_pair['Change7D'].iloc[-1]
        latest_change_30d = df_btc_pair['Change30D'].iloc[-1]

        if not pd.isna(latest_change_1d) :
            latest_change_1d_list.append({'Label': ticker, 'Data': latest_change_1d})
        if not pd.isna(latest_change_7d) :
            latest_change_7d_list.append({'Label': ticker, 'Data': latest_change_7d})
        if not pd.isna(latest_change_30d) :
            latest_change_30d_list.append({'Label': ticker, 'Data': latest_change_30d})

        return_mean = df_btc_pair['Return'].mean(skipna=True)
        return_std = df_btc_pair['Return'].std(skipna=True)
        sharpe_ratio = return_mean/return_std

        # latest_cum_return = df_btc_pair['Cum Return'].iloc[-1]
        return_downside_std = calculate_downside_std(df_btc_pair['Return'])
        # adj_return_per_volatility = latest_cum_return / return_std

        # if not pd.isna(return_mean) :
        #     return_mean_list.append({'Label': ticker, 'Data': return_mean*100})
        # if not pd.isna(return_std) :
        #     return_std_list.append({'Label': ticker, 'Data': return_std})
        if not pd.isna(sharpe_ratio) :
            sharpe_ratio_list.append({'Label': ticker, 'Data': sharpe_ratio})
        # if not pd.isna(adj_return_per_volatility) :
        #     adj_return_per_volatility_list.append({'Label': ticker, 'Data': adj_return_per_volatility})
        if not pd.isna(return_downside_std) :
            sortino_ratio = return_mean/return_downside_std
            sortino_ratio_list.append({'Label': ticker, 'Data': sortino_ratio})

        if len(df_btc_pair['Close']) > 12 :
            momentum = df_btc_pair['Close'].iloc[-2] / df_btc_pair['Close'].iloc[-12]
            momentum_list.append({'Label': ticker, 'Data': momentum})

        if len(df_btc_pair['Close']) > 2 :
            reversal = df_btc_pair['Close'].iloc[-1] / df_btc_pair['Close'].iloc[-2]
            reversal_list.append({'Label': ticker, 'Data': reversal})

        if len(df_btc_pair['Close']) >= 100 and is_show_chart :
            df_btc_pair_weekly = resampling_weekly(df_btc_pair)
            df_btc_pair_weekly = get_indicator(df_btc_pair_weekly)
            show_chart(df_btc_pair_weekly, ticker, 'BTC', 'Weekly')
            show_candlestick(ticker, 'BTC', df_btc_pair_weekly, 'Weekly')

    rsi_list = sorted(rsi_list, key=lambda x: x['RSI_14'])
    rsi_df = pd.DataFrame(rsi_list)
    show_chart_rsi(rsi_df)

    beta_list = sorted(beta_list, key=lambda x: x['Beta'])
    beta_df = pd.DataFrame(beta_list)
    show_chart_beta(beta_df)

    correl_list = sorted(correl_list, key=lambda x: x['Correl'])
    corr_df = pd.DataFrame(correl_list)
    show_chart_correl(corr_df)

    latest_change_1d_list = sorted(latest_change_1d_list, key=lambda x: x['Data'])
    latest_change_1d_list_df = pd.DataFrame(latest_change_1d_list)
    show_chart_change(latest_change_1d_list_df, 'Perubahan 1 Hari Terhadap BTC')

    latest_change_7d_list = sorted(latest_change_7d_list, key=lambda x: x['Data'])
    latest_change_7d_list_df = pd.DataFrame(latest_change_7d_list)
    show_chart_change(latest_change_7d_list_df, 'Perubahan 7 Hari Terhadap BTC')

    latest_change_30d_list = sorted(latest_change_30d_list, key=lambda x: x['Data'])
    latest_change_30d_list_df = pd.DataFrame(latest_change_30d_list)
    show_chart_change(latest_change_30d_list_df, 'Perubahan 30 Hari Terhadap BTC')

    # return_mean_list = sorted(return_mean_list, key=lambda x: x['Data'])
    # return_mean_list_df = pd.DataFrame(return_mean_list)
    # show_chart_return_mean(return_mean_list_df)
    #
    # return_std_list = sorted(return_std_list, key=lambda x: x['Data'])
    # return_std_list_df = pd.DataFrame(return_std_list)
    # show_chart_return_std(return_std_list_df)

    sharpe_ratio_list = sorted(sharpe_ratio_list, key=lambda x: x['Data'])
    sharpe_ratio_list_df = pd.DataFrame(sharpe_ratio_list)
    show_chart_sharpe_ratio(sharpe_ratio_list_df)

    # adj_return_per_volatility_list = sorted(adj_return_per_volatility_list, key=lambda x: x['Data'])
    # adj_return_per_volatility_list_df = pd.DataFrame(adj_return_per_volatility_list)
    # show_chart_adj_return_per_volatility(adj_return_per_volatility_list_df)

    sortino_ratio_list = sorted(sortino_ratio_list, key=lambda x: x['Data'])
    sortino_ratio_list_df = pd.DataFrame(sortino_ratio_list)
    show_chart_sortino_ratio(sortino_ratio_list_df)

    momentum_list = sorted(momentum_list, key=lambda x: x['Data'])
    momentum_list_df = pd.DataFrame(momentum_list)
    show_chart_momentum(momentum_list_df, 'Momentum Pair BTC')

    reversal_list = sorted(reversal_list, key=lambda x: x['Data'])
    reversal_list_df = pd.DataFrame(reversal_list)
    show_chart_reversal(reversal_list_df, 'Reversal Pair BTC')

    excel_file = os.path.join(directory, 'price_bitget-' + get_time_stamp() + '.xlsx')
    with pd.ExcelWriter(excel_file) as writer:
        rsi_df.to_excel(writer, sheet_name='RSI', index=False)
        beta_df.to_excel(writer, sheet_name='Beta', index=False)
        corr_df.to_excel(writer, sheet_name='Correlation', index=False)
        latest_change_1d_list_df.to_excel(writer, sheet_name='Change 1D', index=False)
        latest_change_7d_list_df.to_excel(writer, sheet_name='Change 7D', index=False)
        latest_change_30d_list_df.to_excel(writer, sheet_name='Change 30D', index=False)
        # return_mean_list_df.to_excel(writer, sheet_name='Return Mean', index=False)
        # return_std_list_df.to_excel(writer, sheet_name='Volatility', index=False)
        sharpe_ratio_list_df.to_excel(writer, sheet_name='Sharpe Ratio', index=False)
        sortino_ratio_list_df.to_excel(writer, sheet_name='Sortino Ratio', index=False)
        # adj_return_per_volatility_list_df.to_excel(writer, sheet_name='Adj Return Per Volatility', index=False)
        momentum_list_df.to_excel(writer, sheet_name='Momentum', index=False)
        reversal_list_df.to_excel(writer, sheet_name='Reversal', index=False)

In [ ]:
# symbols = [ '1INCHUSDT', 'AAVEUSDT', 'ACHUSDT', 'ACTUSDT', 'ADAUSDT', 'AEROUSDT', 'AEVOUSDT', 'AI16ZUSDT', 'AIUSDT', 'AIXBTUSDT', 'ALCHUSDT', 'ALEXUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPACAUSDT', 'ALTUSDT', 'ANIMEUSDT', 'ANKRUSDT', 'ANYONEUSDT', 'APEUSDT', 'API3USDT', 'APTUSDT', 'APUUSDT', 'ARBUSDT', 'ARKMUSDT', 'ARUSDT', 'ATHUSDT', 'ATOMUSDT', 'AUCTIONUSDT', 'AVAAIUSDT', 'AVAUSDT', 'AVAXUSDT', 'AXSUSDT', 'BABYDOGEUSDT', 'BBUSDT', 'BCHUSDT', 'BEAMUSDT', 'BERAUSDT', 'BGBUSDT', 'BIGTIMEUSDT', 'BIOUSDT', 'BITCOINUSDT', 'BLASTUSDT', 'BMTUSDT', 'BNBUSDT', 'BNTUSDT', 'BOMEUSDT', 'BONEUSDT', 'BONKUSDT', 'BRETTUSDT', 'BROCCOLIUSDT', 'C98USDT', 'CAKEUSDT', 'CATIUSDT', 'CEEKUSDT', 'CELOUSDT', 'CETUSUSDT', 'CFXUSDT', 'CGPTUSDT', 'CHEEMSUSDT', 'CHESSUSDT', 'CHILLGUYUSDT', 'CKBUSDT', 'CLANKERUSDT', 'CLOREUSDT', 'CLOUDUSDT', 'COMPUSDT', 'COOKIEUSDT', 'COQUSDT', 'COREUSDT', 'COTIUSDT', 'COWUSDT', 'CPOOLUSDT', 'CRVUSDT', 'CTCUSDT', 'CTSIUSDT', 'DARKUSDT', 'DEGENUSDT', 'DMAILUSDT', 'DODOUSDT', 'DOGEUSDT', 'DOGSUSDT', 'DOGUSDT', 'DOTUSDT', 'DRIFTUSDT', 'DYDXUSDT', 'DYMUSDT', 'EIGENUSDT', 'ELONUSDT', 'ENAUSDT', 'ENJUSDT', 'ENSUSDT', 'EOSUSDT', 'ETCUSDT', 'ETHFIUSDT', 'ETHUSDT', 'FARMUSDT', 'FARTCOINUSDT', 'FETUSDT', 'FHEUSDT', 'FIDAUSDT', 'FILUSDT', 'FLOKIUSDT', 'FLUXUSDT', 'FORMUSDT', 'FTTUSDT', 'GALAUSDT', 'GASUSDT', 'GFIUSDT', 'GHIBLIUSDT', 'GLMUSDT', 'GMTUSDT', 'GOATUSDT', 'GRASSUSDT', 'GRIFFAINUSDT', 'GROKUSDT', 'GRTUSDT', 'GUNUSDT', 'GUSDT', 'HBARUSDT', 'HIFIUSDT', 'HIGHUSDT', 'HIPPOUSDT', 'HMSTRUSDT', 'HNTUSDT', 'HTXUSDT', 'HYPEUSDT', 'ILVUSDT', 'IMXUSDT', 'INJUSDT', 'IPUSDT', 'JASMYUSDT', 'JOEUSDT', 'JTOUSDT', 'JUPUSDT', 'KAITOUSDT', 'KASUSDT', 'KMNOUSDT', 'KOMAUSDT', 'L3USDT', 'LADYSUSDT', 'LAYERUSDT', 'LDOUSDT', 'LINKUSDT', 'LISTAUSDT', 'LOKAUSDT', 'LPTUSDT', 'LQTYUSDT', 'LRCUSDT', 'LTCUSDT', 'LUCEUSDT', 'LUNAUSDT', 'LUNCUSDT', 'MANAUSDT', 'MANEKIUSDT', 'MANTAUSDT', 'MASKUSDT', 'MAVIAUSDT', 'MBOXUSDT', 'MELANIAUSDT', 'MERLUSDT', 'METISUSDT', 'MEUSDT', 'MEWUSDT', 'MKRUSDT', 'MOBILEUSDT', 'MOEWUSDT', 'MOGUSDT', 'MOODENGETHUSDT', 'MOODENGUSDT', 'MORPHOUSDT', 'MOTHERUSDT', 'MOVEUSDT', 'MUBARAKUSDT', 'MUMUUSDT', 'MYRIAUSDT', 'MYROUSDT', 'NAKAUSDT', 'NEARUSDT', 'NEIROCTOUSDT', 'NEIROETHUSDT', 'NFPUSDT', 'NKNUSDT', 'NMRUSDT', 'NMTUSDT', 'NOTUSDT', 'NPCUSDT', 'OMUSDT', 'ONDOUSDT', 'ONGUSDT', 'ONTUSDT', 'OPUSDT', 'ORCAUSDT', 'ORDIUSDT', 'PARTIUSDT', 'PAXGUSDT', 'PENDLEUSDT', 'PENGUUSDT', 'PEOPLEUSDT', 'PEPECOINUSDT', 'PEPEUSDT', 'PHAUSDT', 'PHBUSDT', 'PIPPINUSDT', 'PIUSDT', 'PIXELUSDT', 'PLUMEUSDT', 'PNUTUSDT', 'POLUSDT', 'POLYXUSDT', 'PONKEUSDT', 'PORTALUSDT', 'POWRUSDT', 'PRCLUSDT', 'PRIMEUSDT', 'PROPCUSDT', 'PUFFERUSDT', 'PUMPUSDT', 'PYTHUSDT', 'QNTUSDT', 'RACAUSDT', 'RAREUSDT', 'RATSUSDT', 'RAYUSDT', 'RENDERUSDT', 'REZUSDT', 'RFCUSDT', 'RIFSOLUSDT', 'ROAMUSDT', 'ROSEUSDT', 'RSRUSDT', 'RSS3USDT', 'RUNEUSDT', 'SAFEUSDT', 'SAGAUSDT', 'SAMOUSDT', 'SANDUSDT', 'SATSUSDT', 'SDEXUSDT', 'SEIUSDT', 'SHIBUSDT', 'SLERFUSDT', 'SNEKUSDT', 'SNTUSDT', 'SNXUSDT', 'SOLUSDT', 'STGUSDT', 'STORJUSDT', 'STRKUSDT', 'STXUSDT', 'SUIUSDT', 'SUNDOGUSDT', 'SUNUSDT', 'SUPERUSDT', 'SUSDT', 'SUSHIUSDT', 'TAIKOUSDT', 'TAOUSDT', 'THEUSDT', 'TIAUSDT', 'TNSRUSDT', 'TONUSDT', 'TRACUSDT', 'TRBUSDT', 'TREATUSDT', 'TRUMPUSDT', 'TRUUSDT', 'TRXUSDT', 'TURBOUSDT', 'TUTUSDT', 'TWTUSDT', 'UFDUSDT', 'UMAUSDT', 'UNIUSDT', 'UROUSDT', 'USUALUSDT', 'VANRYUSDT', 'VINEUSDT', 'VIRTUALUSDT', 'WENUSDT', 'WHYUSDT', 'WIFUSDT', 'WLDUSDT', 'WUSDT', 'XAIUSDT', 'XAUTUSDT', 'XDCUSDT', 'YGGUSDT', 'ZBCNUSDT', 'ZEREBROUSDT', 'ZETAUSDT', 'ZILUSDT', 'ZKJUSDT', 'ZKUSDT' ]

# symbols = ['DARKUSDT', 'RIFSOLUSDT', 'MOGUSDT', 'SOLUSDT', 'ETHUSDT', 'GAMEUSDT', 'APUUSDT', 'WLDUSDT', 'BNBUSDT']

tickers = [symbol.removesuffix('USDT') for symbol in symbols]

show_multiple_chart(tickers, df_btc_usdt, True)

In [ ]:
def show_matrix_table(matrix_data, chains) :
    df = pd.DataFrame(matrix_data, index=chains, columns=chains)
    df = df.drop(df.index[0], axis=0)
    df = df.drop(df.columns[-1], axis=1)

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')

    table = ax.table(cellText=df.values,
                     rowLabels=df.index,
                     colLabels=df.columns,
                     cellLoc='center',
                     loc='center')

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(df.columns))))

    # plt.show()
    plt.savefig(os.path.join(directory, 'Matrix-Table-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
def show_chain_comparison_chart(chains, is_show_chart=False) :
    full_chain_list = {}

    for i, chain in enumerate(chains) :
        print('Fetching ' + chain + '...')
        df_coin_idr = get_df_from_ticker(chain, time_frame, time_from_ms, time_to_ms)
        full_chain_list[chain] = df_coin_idr

    matrix_data = []
    counter_map = {}
    for i, chain in enumerate(chains):
        row = []
        for j, chain_base in enumerate(chains):
            if i <= j :
                row.append('')
                continue

            print('Processing ' + chain + '/' + chain_base + '...')
            df_coin_idr = full_chain_list[chain]
            df_base_idr = full_chain_list[chain_base]
            df = get_history_btc_pair(chain, df_coin_idr, df_base_idr)
            if is_show_chart :
                show_chart(df, chain, chain_base, 'Daily')
                show_candlestick(chain, chain_base, df, 'Daily')
            return_7d = df['Close'].iloc[-1]/df['Close'].iloc[-7] - 1
            if return_7d >= 0 :
                winner = chain
            else :
                winner = chain_base
            row.append(winner)
            if winner in counter_map :
                counter_map[winner] += 1
            else     :
                counter_map[winner] = 1
            if len(df['Close']) >= 100 and is_show_chart :
                df_weekly = resampling_weekly(df)
                df_weekly = get_indicator(df_weekly)
                show_chart(df_weekly, chain, chain_base, 'Weekly')
                show_candlestick(chain, chain_base, df_weekly, 'Weekly')

        matrix_data.append(row)

    show_matrix_table(matrix_data, chains)
    sorted_map = dict(sorted(counter_map.items(), key=lambda item: item[1], reverse=True))
    print(sorted_map)


In [ ]:
chain_tickers = ['ETH', 'XRP', 'BNB', 'SOL', 'ADA', 'TRX', 'SUI', 'AVAX', 'TON', 'HBAR', 'DOT', 'HYPE', 'APT', 'NEAR', 'ICP', 'POL', 'ARB', 'ATOM', 'S', 'STX', 'OP']

show_chain_comparison_chart(chain_tickers, True)